In [1]:
import os
import sys
sys.path.append('../')
sys.path.append('../datasets')
sys.path.append('../models')
import torch
import torchvision
import torch.backends.cudnn as cudnn
import random
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from importlib import reload
from datasets.three_dim_shapes import ThreeDimShapesDataset
from datasets.small_norb import SmallNORBDataset
from datasets.seq_mnist import SequentialMNIST
import models.seqae as seqae
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from datasets import so3_data as so3d
from einops import rearrange

import csv
import ast



from tqdm import tqdm

if torch.cuda.is_available():
    device = torch.device('cuda')
    cudnn.deterministic = True
    cudnn.benchmark = True
else:
    device = torch.device('cpu')
    gpu_index = -1

    
    
from utils import yaml_utils as yu
import yaml
result_dir = '/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/result'






In [2]:
ls /mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/result/*so3*

/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/result/20220511_so3_exp_0:
C_bd0/  C_bd0id1/  C_bd0id2/  config.yml  log

/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/result/20220611_so3:
config.yml                    snapshot_model_iter_20000
log                           snapshot_model_iter_25000
snapshot_manager_iter_100000  snapshot_model_iter_30000
snapshot_model_iter_10000     snapshot_model_iter_40000
snapshot_model_iter_100000    snapshot_model_iter_50000
snapshot_model_iter_125000    snapshot_model_iter_75000

/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/result/20220611_so3_so3run:
config.yml                    snapshot_model_iter_175000
log                           snapshot_model_iter_200000
snapshot_manager_iter_200000  snapshot_model_iter_25000
snapshot_model_iter_100000    snapshot_model_iter_50000
snapshot_model_iter_125000    snapshot_model_iter_75000
snapshot_model_iter_150000

/mnt/nfs-mnj-hot-01/tmp/masomatics/block_diag/result/20220611_so3_so3run_rcst:
config.yml   

In [3]:
targdir = '20220611_so3_so3run_rcst'
exp_result = os.path.join(result_dir, targdir)
#print(os.listdir(exp_result))
targ_config_path = os.path.join(exp_result, 'config.yml')


with open(targ_config_path, 'r') as f:
    config = yaml.safe_load(f)
    
    
def load_model(model, log_dir, iters):
    print('snapshot_model_iter_{}'.format(iters))
    model.load_state_dict(torch.load(os.path.join(
        log_dir, 'snapshot_model_iter_{}'.format(iters))))

os.listdir(exp_result)

['snapshot_model_iter_200000',
 'config.yml',
 'snapshot_model_iter_25000',
 'snapshot_model_iter_150000',
 'snapshot_model_iter_125000',
 'snapshot_model_iter_50000',
 'snapshot_manager_iter_200000',
 'snapshot_model_iter_100000',
 'snapshot_model_iter_75000',
 'log',
 'snapshot_model_iter_175000']

In [4]:
data = yu.load_component(config['train_data'])
train_loader = DataLoader(data, 
                          batch_size=config['batchsize'],
                          shuffle=True,
                          num_workers=config['num_workers'])
model = yu.load_component(config['model'])
load_model(model, exp_result, 200000)

/usr/local/lib/python3.8/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


snapshot_model_iter_200000


In [6]:
device = 0
model = model.to(device)

scores = [] 
for idx, dat in enumerate(train_loader):
    dat = rearrange(dat, 'b t c d a -> (b t c d) a' ).to(device)
    d0hat =  model.dec(model.enc(dat)).detach().to(device)
    scores.append(seqae.r2_score(d0hat.to('cpu'), dat.to('cpu')))
    
scores = torch.tensor(scores)
print(torch.mean(scores))

tensor(-11092.7831, dtype=torch.float64)


In [8]:
dat

tensor([[-0.6631,  0.1084, -1.0163,  ...,  0.7172,  0.2431,  1.0904],
        [-0.6144,  0.1436, -0.9959,  ...,  0.6797,  0.2190,  1.0994],
        [-0.5688,  0.1794, -0.9756,  ...,  0.6322,  0.2036,  1.0948],
        ...,
        [-0.1425,  0.5347, -1.1160,  ..., -0.0811,  0.5421,  1.0828],
        [ 1.0720, -0.5884,  0.3470,  ...,  0.5404,  0.1361,  0.6540],
        [ 0.4324, -0.7878,  0.8426,  ...,  0.4858, -0.2519,  0.0097]],
       device='cuda:0')

In [9]:
d0hat

tensor([[-0.7652, -0.4639,  1.7995,  ..., -0.6526,  0.4311, -0.2648],
        [-0.7649, -0.4634,  1.7995,  ..., -0.6527,  0.4306, -0.2651],
        [-0.7647, -0.4629,  1.7993,  ..., -0.6526,  0.4301, -0.2655],
        ...,
        [-0.7630, -0.4578,  1.7998,  ..., -0.6419,  0.4196, -0.2494],
        [-0.7612, -0.4627,  1.8105,  ..., -0.6455,  0.4177, -0.2538],
        [-0.7649, -0.4604,  1.8115,  ..., -0.6535,  0.4180, -0.2607]],
       device='cuda:0')

In [11]:
torch.mean(dat, axis=0)

tensor([ 0.0727,  0.0106,  0.0445,  0.3173,  0.2692,  0.0076,  0.2875, -0.3427,
         0.4088, -0.0959, -0.5769, -0.0769,  0.2160, -0.1123,  0.1609,  0.0405,
         0.0057, -0.3580,  0.3251, -0.0583, -0.1849, -0.0815, -0.0157,  0.0281,
        -0.1199, -0.0267, -0.2218,  0.0377,  0.4011, -0.0633, -0.0341,  0.0008,
         0.1554,  0.0245,  0.0387, -0.0453, -0.0267, -0.0436,  0.0895,  0.0022,
         0.2128, -0.2091, -0.2406, -0.0892,  0.1419,  0.3245, -0.0577, -0.2936,
        -0.2338,  0.0163,  0.0485,  0.1300,  0.4111, -0.1679,  0.0232,  0.1191,
        -0.0314,  0.0764,  0.2522,  0.2792], device='cuda:0')

In [20]:
model.dec.network[2]

Invertible_Resblock_Fc(
  (bottleneck_block): Sequential(
    (0): Linear(in_features=60, out_features=60, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=60, out_features=60, bias=True)
  )
)

In [21]:
model.dec.network

Sequential(
  (0): Invertible_Resblock_Fc(
    (bottleneck_block): Sequential(
      (0): Linear(in_features=60, out_features=60, bias=True)
      (1): ELU(alpha=1.0)
      (2): Linear(in_features=60, out_features=60, bias=True)
    )
  )
  (1): Invertible_Resblock_Fc(
    (bottleneck_block): Sequential(
      (0): Linear(in_features=60, out_features=60, bias=True)
      (1): ELU(alpha=1.0)
      (2): Linear(in_features=60, out_features=60, bias=True)
    )
  )
  (2): Invertible_Resblock_Fc(
    (bottleneck_block): Sequential(
      (0): Linear(in_features=60, out_features=60, bias=True)
      (1): ELU(alpha=1.0)
      (2): Linear(in_features=60, out_features=60, bias=True)
    )
  )
)